In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from transformers import TextStreamer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model_id = "google/gemma-2-2b-it"

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4bit_compute_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
)

chat = [
    { "role": "user", "content": "Random prompt."},
]
question = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

question = tokenizer(question, return_tensors="pt").to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(question)

{'input_ids': tensor([[     2,      2,    106,   1645,    108,  13020,  18335, 235265,    107,
            108,    106,   2516,    108]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [11]:
_ = model.generate(**question, streamer=streamer,
                            pad_token_id=tokenizer.eos_token_id,
                            temperature=0.1,
                            max_length=2048,
                            do_sample=True,
                            top_p=0.9,
                            repetition_penalty=1.25)

## Prompt: 

**Imagine you are a time traveler from the year 2347, visiting Earth in 2023.** What is your first impression of this place?  What do you find surprising or fascinating about it compared to what you expected based on historical records and projections for our future?


Let me know how I can help with any details! 😊 
<end_of_turn>


In [8]:
decode_text = tokenizer.decode(_[0], skip_special_tokens=True)

In [9]:
print(decode_text)

user
Random prompt.
model
## Prompt: 

**Imagine you are a time traveler from the year 2347, visiting Earth in 2023.** What is your first impression of this world?  


Let me know what kind of details and thoughts come to mind! I'm excited to see how we can explore this fascinating scenario together. 😊 🚀 🌎 🤔 



In [10]:
question["input_ids"][0]

tensor([     2,      2,    106,   1645,    108,  13020,  18335, 235265,    107,
           108,    106,   2516,    108], device='cuda:0')